<a href="https://colab.research.google.com/github/Kiara-Wu/06-CHL5230-F23/blob/main/random%20forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df = pd.read_csv('/content/datathon4.csv')
df1 = pd.DataFrame(df)
df1 = df1.drop(columns=['encounter_id','patient_id','hospital_id','apache_2_diagnosis', 'apache_3j_diagnosis'])

string_columns = df1.select_dtypes(include=['object']).columns
# Drop the selected string columns
df1 = df1.drop(columns=string_columns)
df1 = df1[df1['hepatic_failure']==1]

df1.head()

,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,...,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
35,0,71.0,38.497016,0,177.8,100,0.000694,0,121.7,NaN,...,0.10,0.04,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
49,0,67.0,28.690874,0,177.8,82,0.013889,0,90.7,2.3,...,0.13,0.07,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
103,0,71.0,35.078125,0,160.0,97,0.000694,0,89.8,2.6,...,0.09,0.05,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
113,0,48.0,25.968033,0,175.3,98,0.009722,0,79.8,2.0,...,0.04,0.01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
138,0,55.0,23.570601,0,172.7,103,0.013889,0,70.3,2.9,...,0.41,0.28,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
df1.columns

Index(['hospital_death', 'age', 'bmi', 'elective_surgery', 'height', 'icu_id',
       'pre_icu_los_days', 'readmission_status', 'weight', 'albumin_apache',
       ...
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis'],
      dtype='object', length=173)

In [25]:
# checking the data types
df1.dtypes

hospital_death                   int64
age                            float64
bmi                            float64
elective_surgery                 int64
height                         float64
                                ...   
hepatic_failure                float64
immunosuppression              float64
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
Length: 173, dtype: object

In [26]:
import matplotlib.pyplot as plt
# Get the counts of each unique value in the 'suicide_level' column
counts = df1['hospital_death'].value_counts()

# Define a custom color sequence for the pie chart
colors = ['#1c3a73', '#7cb1c2']

# Create a pie chart using the Plotly Express library
# - names: This denotes the unique labels in the 'suicide_level' column
# - values: This denotes the count of each unique label in the 'suicide_level' column
# - hole: This will create a donut-shaped pie chart. A value of 0.5 means the hole is 50% of the total pie.
# - title: The main title for the pie chart
# - color_discrete_sequence: Specifies the color sequence for the pie slices
fig = px.pie(
    names=counts.index,
    values=counts.values,
    hole=0.5,
    title='Distribution of Classes in "hospital_death"',
    color_discrete_sequence=colors
)

# Update the layout and appearance of the pie chart
fig.update_layout(
    margin=dict(t=50, b=0, l=0, r=0),  # Adjust the margins of the plot
    legend=dict(  # Define legend properties
        orientation="v",
        yanchor="top",
        y=0.5,
        xanchor="left",
        x=1.05,
        title='Class/ASD'
    ),
    title=dict(  # Define title properties
        text='Distribution of Classes in "hospital_death"',
        x=0.45,
        xanchor='center',
        y=0.95,
        yanchor='top',
        font=dict(size=16)
    ),
    width=500,  # Set the width of the figure
    height=350   # Set the height of the figure
)

# Display the pie chart
fig.show()

**manipulating missing values by filling with mean of columns**

In [27]:
df1.isnull().sum()

hospital_death                  0
age                            10
bmi                            37
elective_surgery                0
height                         10
                               ..
hepatic_failure                 0
immunosuppression               0
leukemia                        0
lymphoma                        0
solid_tumor_with_metastasis     0
Length: 173, dtype: int64

In [28]:
# Impute missing values with the mean of the column
df1 = df1.fillna(df1.mean())

df1.isnull().sum()

hospital_death                 0
age                            0
bmi                            0
elective_surgery               0
height                         0
                              ..
hepatic_failure                0
immunosuppression              0
leukemia                       0
lymphoma                       0
solid_tumor_with_metastasis    0
Length: 173, dtype: int64

In [29]:
description = df1.describe()
print(description)


       hospital_death          age          bmi  elective_surgery  \
count     1182.000000  1182.000000  1182.000000       1182.000000   
mean         0.181049    59.421502    29.048343          0.067682   
std          0.385222    12.852968     7.618854          0.251306   
min          0.000000    18.000000    14.844926          0.000000   
25%          0.000000    51.000000    24.013462          0.000000   
50%          0.000000    59.000000    27.982202          0.000000   
75%          0.000000    68.000000    32.430628          0.000000   
max          1.000000    89.000000    67.814990          1.000000   

            height       icu_id  pre_icu_los_days  readmission_status  \
count  1182.000000  1182.000000       1182.000000              1182.0   
mean    170.627551   484.956007          1.167752                 0.0   
std      10.283467   209.615897          3.087557                 0.0   
min     137.200000    82.000000         -0.090278                 0.0   
25%     162.6

**Feature Selection**\
Method 1: correlation matrix\
method 2: Lasso Regression

In [30]:


# Calculate correlations
correlations = df1.corr()
# Select top N features based on correlation
top_features = correlations['hospital_death'].abs().nlargest(10).index
print(top_features)


Index(['hospital_death', 'd1_lactate_min', 'd1_lactate_max', 'd1_inr_max',
       'h1_inr_max', 'd1_mbp_min', 'apache_4a_icu_death_prob',
       'd1_mbp_noninvasive_min', 'd1_inr_min', 'h1_inr_min'],
      dtype='object')


In [31]:

X = df1.drop(columns=['hospital_death'])
y = df1[['hospital_death']]

from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1.0)  # You can adjust the alpha parameter for regularization strength

# Fit the model on your data
lasso.fit(X, y)

# You can access the coefficients of the selected features using lasso.coef_
selected_feature_indices = np.where(lasso.coef_ != 0)[0]
selected_features = X.columns[selected_feature_indices]
selected_features


Index(['icu_id', 'glucose_apache', 'map_apache', 'urineoutput_apache',
       'd1_heartrate_max', 'd1_mbp_min', 'd1_sysbp_min', 'h1_sysbp_min',
       'd1_bun_max', 'd1_arterial_po2_max', 'd1_pao2fio2ratio_max',
       'd1_pao2fio2ratio_min'],
      dtype='object')

In [11]:
# random forest libs
import pandas as pd
import numpy as np
import warnings
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
# from sklearn.metrics import RocCurveDisplay, auc, roc_curve
from sklearn.metrics import roc_auc_score, accuracy_score
import plotly.express as px
#from category_encoders import OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


warnings.filterwarnings('ignore')

**Using features selected by Lasso Regression to predict hospital death**

In [35]:
df2 = df1[['icu_id', 'glucose_apache', 'map_apache', 'urineoutput_apache',
       'd1_heartrate_max', 'd1_mbp_min', 'd1_sysbp_min', 'h1_sysbp_min',
       'd1_bun_max', 'd1_arterial_po2_max', 'd1_pao2fio2ratio_max',
       'd1_pao2fio2ratio_min','hospital_death']]
df2.head()



,icu_id,glucose_apache,map_apache,urineoutput_apache,d1_heartrate_max,d1_mbp_min,d1_sysbp_min,h1_sysbp_min,d1_bun_max,d1_arterial_po2_max,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,hospital_death
35,100,198.0,137.0,1303.984794,96.0,76.0,114.0,136.0,16.0,159.955445,279.685685,215.871653,0
49,82,147.0,64.0,1303.984794,110.0,64.0,75.0,104.0,8.0,373.000000,279.685685,215.871653,0
103,97,181.0,70.0,1303.984794,116.0,58.0,95.0,130.0,24.0,159.955445,279.685685,215.871653,0
113,98,159.0,57.0,1303.984794,140.0,57.0,80.0,101.0,37.0,159.955445,279.685685,215.871653,0
138,103,108.0,61.0,1303.984794,103.0,88.0,106.0,106.0,14.0,344.000000,344.000000,312.500000,0


**Random Forest Classifier**

In [36]:
# Define the column that will be used as the target for modeling or analysis
target_column = 'hospital_death'

from sklearn.model_selection import train_test_split

# Split data into features (X) and the target variable (y)
X = df2.drop(columns=[target_column])
y = df2[target_column]

#from sklearn.ensemble import RandomForestRegressor  # For regression tasks
from sklearn.ensemble import RandomForestClassifier  # For classification tasks

# Create a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)  # Example hyperparameters

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model to your training data
model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [37]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8143459915611815


**Hyperparameter Tuning with Grid Search**

In [38]:
# Define the hyperparameter values that should be tested
param_grid = {
    'n_estimators': [100],
    'max_depth': [5,10,15],
    'min_samples_leaf': [15,20,25]
}
# Initializing stratified K-fold cross-validation. This ensures each fold has the same proportion of target labels as the whole dataset.
#skf = StratifiedKFold(n_splits=5)

# Setting up GridSearchCV to search through the parameter space.
# It will evaluate the model for each combination of hyperparameters provided using stratified K-fold cross-validation.
#random_search = GridSearchCV(model, param_grid=param_dist, cv=skf)

grid_search = GridSearchCV(estimator= model, param_grid=param_grid, scoring='accuracy', cv=2)
grid_search.fit(X_train, y_train)

#best_params = grid_search.best_params_

#best_model = RandomForestClassifier(**best_params)
#best_model.fit(X_train, y_train)
#accuracy = best_model.score(X_test, y_test)

results = pd.DataFrame(grid_search.cv_results_)

# Filtering to get the rows with the best performing hyperparameters.
results[results['rank_test_score'] == 1]



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.192823,0.001418,0.011965,0.000214,5,15,100,"{'max_depth': 5, 'min_samples_leaf': 15, 'n_es...",0.839323,0.84322,0.841272,0.001948,1
3,0.197180,0.009242,0.012315,0.000077,10,15,100,"{'max_depth': 10, 'min_samples_leaf': 15, 'n_e...",0.839323,0.84322,0.841272,0.001948,1
6,0.184565,0.001524,0.012406,0.000232,15,15,100,"{'max_depth': 15, 'min_samples_leaf': 15, 'n_e...",0.839323,0.84322,0.841272,0.001948,1


**Using Optimal Hyperparameters to Train and Evaluate the Model**

In [39]:
# Using Optimal Hyperparameters to Train and Evaluate the Model

# "best_params" refers to the set of hyperparameters that resulted in the highest performance
# (e.g., highest accuracy, lowest error) on the validation data during the hyperparameter tuning process.

#"best_model" is the machine learning model that was trained using the hyperparameters found in "best_params."
best_params = grid_search.best_params_
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
accuracy = best_model.score(X_test, y_test)
accuracy


0.8354430379746836

In [40]:
# mean cross-validated score of the best estimator found during the hyperparameter tuning process
best_score = grid_search.best_score_
grid_search_results = grid_search.cv_results_
best_score


0.8412719031067474

**Use features selected by correlation matrix**\
**Repeat random forest**

In [41]:
df3 = df1[['hospital_death', 'd1_lactate_min', 'd1_lactate_max', 'd1_inr_max',
       'h1_inr_max', 'd1_mbp_min', 'apache_4a_icu_death_prob',
       'd1_mbp_noninvasive_min', 'd1_inr_min', 'h1_inr_min']]
df3.head()

,hospital_death,d1_lactate_min,d1_lactate_max,d1_inr_max,h1_inr_max,d1_mbp_min,apache_4a_icu_death_prob,d1_mbp_noninvasive_min,d1_inr_min,h1_inr_min
35,0,3.331377,4.458717,2.050625,2.050625,76.0,0.04,76.0,1.870993,1.870993
49,0,3.331377,4.458717,1.100000,1.100000,64.0,0.07,64.0,1.100000,1.100000
103,0,3.331377,4.458717,2.050625,2.050625,58.0,0.05,58.0,1.870993,1.870993
113,0,3.331377,4.458717,1.400000,1.400000,57.0,0.01,57.0,1.400000,1.400000
138,0,3.331377,4.458717,2.050625,2.050625,88.0,0.28,88.0,1.870993,1.870993


In [42]:
# Define the column that will be used as the target for modeling or analysis
target_column = 'hospital_death'

from sklearn.model_selection import train_test_split

# Split data into features (X) and the target variable (y)
X = df3.drop(columns=[target_column])
y = df3[target_column]

#from sklearn.ensemble import RandomForestRegressor  # For regression tasks
from sklearn.ensemble import RandomForestClassifier  # For classification tasks

# Create a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)  # Example hyperparameters

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model to your training data
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8312236286919831


In [43]:
# Define the hyperparameter values that should be tested
param_grid = {
    'n_estimators': [100],
    'max_depth': [5,10,15],
    'min_samples_leaf': [15,20,25]
}
# Initializing stratified K-fold cross-validation. This ensures each fold has the same proportion of target labels as the whole dataset.
#skf = StratifiedKFold(n_splits=5)

# Setting up GridSearchCV to search through the parameter space.
# It will evaluate the model for each combination of hyperparameters provided using stratified K-fold cross-validation.
#random_search = GridSearchCV(model, param_grid=param_dist, cv=skf)

grid_search = GridSearchCV(estimator= model, param_grid=param_grid, scoring='accuracy', cv=2)
grid_search.fit(X_train, y_train)

#best_params = grid_search.best_params_

#best_model = RandomForestClassifier(**best_params)
#best_model.fit(X_train, y_train)
#accuracy = best_model.score(X_test, y_test)

results = pd.DataFrame(grid_search.cv_results_)

# Filtering to get the rows with the best performing hyperparameters.
results[results['rank_test_score'] == 1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
4,0.176543,0.001339,0.012229,0.000712,10,20,100,"{'max_depth': 10, 'min_samples_leaf': 20, 'n_e...",0.854123,0.860169,0.857146,0.003023,1
7,0.179967,0.007175,0.011534,0.000050,15,20,100,"{'max_depth': 15, 'min_samples_leaf': 20, 'n_e...",0.854123,0.860169,0.857146,0.003023,1


In [44]:
# Using Optimal Hyperparameters to Train and Evaluate the Model

# "best_params" refers to the set of hyperparameters that resulted in the highest performance
# (e.g., highest accuracy, lowest error) on the validation data during the hyperparameter tuning process.

#"best_model" is the machine learning model that was trained using the hyperparameters found in "best_params."
best_params = grid_search.best_params_
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
accuracy = best_model.score(X_test, y_test)
accuracy

0.8649789029535865

In [45]:
# mean cross-validated score of the best estimator found during the hyperparameter tuning process
best_score = grid_search.best_score_
grid_search_results = grid_search.cv_results_
best_score


0.8571460565449529